### Evolving the matter field
***
In this tutorial we will learn how to evolve the initial conditions using both Eularian and Lagrangian perturbation theory.

We will use the functionality of `21cmFAST` that is discribed in detail here
[https://21cmfast.readthedocs.io/en/latest/tutorials/coeval_cubes.html](https://21cmfast.readthedocs.io/en/latest/tutorials/coeval_cubes.html)
and take advantage of some plotting analysis routines in the `tuesday` package described here
[https://tuesday-eor.readthedocs.io/en/latest/reference/index.html](https://tuesday-eor.readthedocs.io/en/latest/reference/index.html)

### Begin initialization

In [ ]:
### This is for google collab or any machine that has not installed 21cmFAST yet
!sudo apt-get install libfftw3-dev libgsl-dev
%pip install pip --upgrade
%pip install 21cmFAST==4.0.0b0 tuesday-eor

In [ ]:
# Standard initialization for 21cmFAST tutorials
import py21cmfast as p21c
import numpy as np  
import matplotlib.pyplot as plt
from py21cmfast import plotting
from py21cmfast.wrapper import cfuncs as cf # C function hooks
from astropy import units
import astropy.constants as const
import astropy.cosmology as cosmo
from tempfile import mkdtemp
from pathlib import Path
import cv2
import os
import math as m
import tuesday.core as tools

cache = p21c.OutputCache("./Cache") # Create a temporary directory for the cache


### End initialization
***

Let us create **initial conditions** (or load from previous tutorial).  Look up `InputParameters.from_template`

In [ ]:
### Create and visualize the initial conditions (standardize this)

# set up the global parameters
DIM = 600
N_THREADS = 2
HII_DIM = 200
BOX_LEN = 300
inputs = p21c.InputParameters.from_template(
    "simple", random_seed=1234
).evolve_input_structs(BOX_LEN=BOX_LEN, DIM=DIM, HII_DIM=HII_DIM, N_THREADS=N_THREADS) #adapt N_THREADS to your machine

# make initial conditions (or load them from previous run)
initial_conditions = p21c.compute_initial_conditions(inputs=inputs, cache=cache,  regenerate=False, write=True) # set regenerate=True to force a new run

As we did in the previous lesson, let's use 'plot_coeval_slice' from ['tuesday.core'](https://github.com/21cmfast/tuesday/blob/main/docs/tutorials/computing_and_plotting_power_spectra.ipynb) to visualize a 2D slice of the initial conditions.

*Note: the tuesday-eor package would like all quantities to have units explicitly defined. Use [astropy.units](https://docs.astropy.org/en/stable/units/index.html)*

In [ ]:
### visualize the initial conditions
# plot a 2D slice
tools.plot_coeval_slice(initial_conditions.hires_density.value*units.dimensionless_unscaled, 
                        BOX_LEN * units.Mpc, transform2slice = tools.coeval2slice_z(idx=0), 
                        title="$\\delta_m$", cmap="viridis", vmin=-5, vmax=5)

Now let's try **evolving** the initial conditions using **Eularian linear perturbation theory**.

Recall that the output is the **growing mode of the overdensity field linearly-extrapolated to z=0**:
\begin{equation}
\delta_m \equiv \rho/\bar{\rho}-1.
\end{equation}

So the Eularian evolution would just scale ALL modes by the **growth factor**, $D(z)$  

You can use the `cf.get_growth_factor(inputs, redshift)` for the growth_factor.<br>
Let's look at the following redshifts: `redshifts = [100, 20, 10, 5, 0]', and remember to set the same color bar range using 'vmin' and 'vmax' (e.g. from -5 to 5) so that the different redshift outputs can be compared more easily.


In [ ]:
# let us loop over redshifts and update the plots above
redshifts = [100, 20, 10, 5, 2, 0]
for z in redshifts:
    D = cf.get_growth_factor(inputs=inputs, redshift=z) # compute the linear growth factor
    tools.plot_coeval_slice(initial_conditions.hires_density.value*D*units.dimensionless_unscaled, 
                        BOX_LEN * units.Mpc, transform2slice = tools.coeval2slice_z(idx=0), 
                        title=f"$\\delta_m$ at z={z:.2f} (D(z) = {D:.2f})",
                        cmap="viridis", vmin=-5, vmax=5)

*Can we see the growth of non-linear structure?  How would you expect the true Universe to evolve?*

Eularian perturbation theory (perturbing the overdensity, $\delta_m$) only changes the contrast independently of the scale.  
There is no mass transfer between cells, no cosmic web...

Let's get more quantitiative and use the `calculate_ps_coeval` and `plot_pdf` functions in [tuesday](https://tuesday-eor.readthedocs.io/en/latest/reference/index.html) to plot the spherically-averaged power spectra, and histograms computed on the cell size at these redshifts.  You can make a 2 planel plot using `fig, ax = plt.subplots(1, 2, figsize=(16, 6))` for example.

To save time, let's use  the lowres ICs density field: 
`initial_conditions.lowres_density.value`.

In [ ]:
#define the plot format
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

#loop over redshifts and use indexing to change colors
for idx, z in enumerate(redshifts):
   D = cf.get_growth_factor(inputs=inputs, redshift=z) # compute the linear growth factor
   delta_x = initial_conditions.lowres_density.value*D*units.dimensionless_unscaled
   ps1d, ps2d = tools.calculate_ps_coeval(delta_x, 
      BOX_LEN * units.Mpc, calc_2d=True, calc_1d=True, deltasq=True)
   tools.plot_power_spectrum(ps1d, ax=ax[0], color=f"C{idx}", legend=z,logx=True, 
                             logy=True, ylabel='$\\Delta^2(k)$')
   tools.plot_pdf(delta_x, ax=ax[1], hist_kwargs={'bins':100, 'color': f"C{idx}", 
                  "histtype": "step", "range":(-5, 5), "density":True})
ax[0].set_title("Power Spectrum of $\\delta_m$")
ax[1].set_title("PDF of $\\delta_m$")
ax[1].set_ylabel("PDF")
ax[1].legend([f"z={z}" for z in redshifts])

*What qualitative behavior do you note?*

*Why are some power spectrum bins empty?*

*How would you expect these quantities to evolve in the real Universe as structures form (i.e. scales become non-linear)?*

***

Luckily, we can use the powerful **Spherical Collapse Model** to connect the **linear matter overdensity** to **halos**.


Let's start by visualizing where are the cells whose overdensities cross the `Spherical Collapse` barrier:
\begin{equation}
\nonumber \delta_{\rm crit}(z) = 1.686 / D(z)
\end{equation}
, where we followed convensition and defined the critical threshold to be a function of redshift, operating on the z=0 matter field.


Let's use the snapshot at z=2 and paint cells in red that have an overdensity greater than the critical value.  use `plot_coval_slice` again, and `scatter` to overplot the marked cells (remember to strech so that the axis are the same).

In [ ]:
# make the plots above
z= 2
D = cf.get_growth_factor(inputs=inputs, redshift=z) # compute the linear growth factor
ax = tools.plot_coeval_slice(initial_conditions.hires_density.value*D*units.dimensionless_unscaled, 
                         BOX_LEN * units.Mpc, transform2slice = tools.coeval2slice_z(idx=0), 
                         title="$\\delta_m$", cmap="viridis", vmin=-5, vmax=5)
plt.title(f"$\\delta_m$ at z={z:.2f} (linear growth factor = {D:.2f})")
# Let's find the cells that are above the threshold
X, Y = np.nonzero(initial_conditions.hires_density.value[:,:,0] > (1.686 / D)) # find the cells that are above the threshold
ax.scatter( X*BOX_LEN/DIM, Y*BOX_LEN/DIM, c='r', s=0.01, alpha=0.6) # plot the cells that are above the threshold

*Compare the red dots to the linear matter field from above...  What do you notice?*

***
Let's get a little more quantitative, and compute the *collapsed fraction*, defined in class.

We will compute it analytically and numerically from our realization of the linear matter field, and compare the two results.

Let's start by computing the **Lagrangian mass of a cell**.  This corresponds to our simulation's "resolution limit".  Recall that the initial conditions are defined on a grid whose side length is:<br>
`BOX_LEN / DIM`

*What is the mass corresponding to this cell?*

We will use this value later on, so let us call this cell mass **Mmin**

*(Note: that the present-day critical density can be obtained from the astropy object `inputs.cosmo_params.cosmo.critical_density0`, and the present-day matter density is found in `inputs.cosmo_params.OMm`*

In [ ]:
###  Compute the mass in a cell
rho_crit = inputs.cosmo_params.cosmo.critical_density0.to(units.Msun/units.Mpc**3)  # in Msun Mpc^-3
OMm = inputs.cosmo_params.OMm # matter density parameter
cell_len = BOX_LEN / DIM * units.Mpc # size of a cell in Mpc
Mmin = rho_crit * OMm * cell_len**3
print(f"Lagrangian mass in a cell = {Mmin:.2e}")

Now, using this `Mmin`, compute the Press-Schecture collapse fraction (i.e. using the scale-independent barrier from above) at, say, z=2:
\begin{equation}
\nonumber f_{\rm coll}(>M_{\rm min}, z=2) = {\rm erfc}\left[ \frac{\delta_{\rm crit}(z)}{\sqrt{2} \sigma(M_{\rm min})} \right] ~ ,
\end{equation}

To obtain the standard deviation of the matter field smoothed on scale $M_{\rm min}$, you can use the built in 21cmFAST function:<br>
`sigma, dsigmasqdm = cf.evaluate_sigma(inputs, masses)`

*Note: routines in cfuncs want the unitless mass, so remember to do `masses=Mmin.to(units.Msun).value`*

In [ ]:
# compute the std of the matter field at z=2 on scale of the DIM grid
z=2
sigma, dsigmasq = cf.evaluate_sigma(inputs=inputs, masses=Mmin.to(units.Msun).value)
D = cf.get_growth_factor(inputs=inputs, redshift=z)
delta_crit = 1.686 / D
f=m.erfc(delta_crit / (m.sqrt(2) * sigma))
print(f"f_coll(M_halo>{Mmin:.0e}, z={z:.2f}) = {f:.2f}")

Now compute the collapse fraction at z=2 directly from the field in `initial_conditions.hires_density.value`

In [ ]:
# compute the fraction of cells with delta > 1.686 and multiply by 2
f = 2*np.sum(initial_conditions.hires_density.value > (1.686 / D)) / initial_conditions.hires_density.value.size
print(f"f_coll(M_halo>{Mmin:.0e}, z={z:.2f}) = {f:.2f}")

*Does the analytic estimate agree with the one using the linear matter field?*

<div class="alert alert-block alert-warning">
Why did we need a factor of two when computing the collapsed fraction from the maps?

The two results above did not agree exactly though...  

*Why could that be?*
*How could you test your theory?*

***

Ok, now let us treat the field as particles and perturb their **DISPLACEMENT** (instead of the overdensity that we perturbed using Eularian PT).  This is called **Lagrangian perturbation theory (LPT)**.

21cmFAST has a function called `PERTURB_FIELD` that we can use for this purpose.  Let's use `PERTURB_FIELD` to plot the density field at the same redshifts as above.

In [ ]:
# loop over redshifts and call perturb_field.  Try saving LPT fields in a dictionary
LPT_fields = {}
for z in redshifts:
    perturbed_field = p21c.perturb_field(redshift=z, initial_conditions=initial_conditions)
    #plt.imshow(perturbed_field.density.value[:,:,0], vmin=-5, vmax=5) # plot 2D slice; to be replaced
    plotting.coeval_sliceplot(perturbed_field, "density", cbar_label='$\\delta_m$', cmap="viridis", vmin=-5, vmax=5);
    plt.title(f"$\\delta_m$ at z={z:.2f}")
    plt.show()
    LPT_fields[z] = perturbed_field # store the LPT fields in a dictionary
   

*How does Lagrangian PT theory compare with Eularian?  Do you see a "web"? "voids"?*

*Note that in oder to define a matter overdensity, we used our high resolution `DIM` grid as "particles", moved them, and then re-bined them to a lower resolution `HII_DIM` grid. Thus these fields are lower resolution that the ones we saw earlier.  They are at the same resolution as our `lowres_density` ICs.*


To visualize better what Lagrangian PT is doing, let us remake the last (i.e. z=0) plot, but now overplotting the linear velocity vectors.  We will use the `quiver` and `meshgrid` functions from `matplotlib.pyplot`.

In [ ]:
X = np.arange(0, HII_DIM, 10)
X, Y = np.meshgrid(X, X)
ax = tools.plot_coeval_slice(LPT_fields[0].density.value*units.dimensionless_unscaled, 
                         BOX_LEN * units.Mpc, transform2slice = tools.coeval2slice_z(idx=0), 
                         title="$\\delta_m$", cmap="viridis", vmin=-5, vmax=5)
#plt.imshow(LPT_fields[0].density.value[:,:,0].T, vmin=-5, vmax=5, origin="lower") # plot 2D slice; to be replaced
ax.quiver(X*BOX_LEN/HII_DIM, Y*BOX_LEN/HII_DIM, initial_conditions.lowres_vx.value[X,Y,0], initial_conditions.lowres_vy.value[X,Y,0], 
        color='r', width=0.006, headwidth=4, alpha=0.5),

*How are arrows pointing in general?*
***

<div class="alert alert-block alert-info">
FYI: The `PERTURB_FIELD` function actually allows us to use different perturbation theories to evolve the density field.  We can chose from Eularian linear evolution, first order LPT (i.e. the Zel'dovich Approximation), and second order LPT (2LPT).
</br>

<b>Physics Bonus:</b> Find the `perturb_field` function in the `src` directory of `21cmFAST` and see how Eularian PT and the ZA are implemented, comparing to our lecture notes.
</div>

Now let us remake the plot of the power spectra and PDF but using our LPT fields.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
#loop over redshifts and use indexing to change colors
for idx, z in enumerate(redshifts):
   delta_x = LPT_fields[z].density.value*units.dimensionless_unscaled
   ps1d, ps2d = tools.calculate_ps_coeval(delta_x, 
      BOX_LEN * units.Mpc, calc_2d=True, calc_1d=True, deltasq=True)
   tools.plot_power_spectrum(ps1d, ax=ax[0], color=f"C{idx}",
                             legend=z,logx=True, logy=True, ylabel='$\\Delta^2(k)$')
   tools.plot_pdf(delta_x, ax=ax[1], hist_kwargs={'bins':100, 'color': f"C{idx}", 
                  "histtype": "step", "range":(-2, 2), "density":True})
ax[0].set_title("Power Spectrum of $\\delta_m$")
ax[1].set_title("PDF of $\\delta_m$")
ax[1].legend([f"z={z}" for z in redshifts])
ax[1].set_ylabel("PDF")

*What do you see?  How does this compare with the Eularian linear evolution?*

***
<div class="alert alert-block alert-info">
<b>Movie Bonus:</b>
You can make a movie of your Universe evolving with time!  Let's plot the 2D slices through the matter fields evolved with 2LPT, for logarithmic redshift evolution.  We will save the plots to a local directory as images and then use `OPENCV` to combine them into a movie.

In [ ]:
# make a 'Frames' directory
frames_folder = './Frames'
video_name = 'matter_evolution_2LPT.avi'
Path(frames_folder).mkdir(parents=True, exist_ok=True)
os.system("rm "+frames_folder+"/*.*"); # remove existing files
os.system("rm "+video_name); # remove existing files

In [ ]:
# loop over redshifts and make plots
num_frames = 30
for z in np.logspace(1.3, -0.5, num_frames):
    perturbed_field = p21c.perturb_field(redshift=z, initial_conditions=initial_conditions, cache=cache, regenerate=False, write=True)
    ax = tools.plot_coeval_slice(perturbed_field.density.value*units.dimensionless_unscaled, 
                         BOX_LEN * units.Mpc, transform2slice = tools.coeval2slice_z(idx=0), 
                         title="$\\delta_m$", cmap="viridis", vmin=-5, vmax=5)
    ax.set_title(f"$\\delta_m$ at z={z:.2f}");
    plt.savefig(f"./Frames/matter_overdensity_z{z:06.2f}.png");
    plt.close();


In [ ]:
# make a movie from the frames
images = [img for img in sorted(os.listdir(frames_folder))[::-1] if img.endswith(".png")]
frame = cv2.imread(os.path.join(frames_folder, images[0]))
height, width, layers = frame.shape

fps=5
video = cv2.VideoWriter(video_name, 0, fps, (width,height))
for image in images:
    video.write(cv2.imread(os.path.join(frames_folder, image)))

video.release()